In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ib_insync import *
import pandas as pd
from IPython.display import display, clear_output
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from keras.src.callbacks import EarlyStopping
import random
import numpy as np
import tensorflow as tf
from keras.api.layers import Dense, Dropout
from keras.api.models import Sequential
from keras.api.regularizers import l1, l2
from keras.api.optimizers import Adam
util.startLoop()

In [23]:
import keras
import pickle

model = keras.models.load_model('daily_v70.keras')
params = pickle.load(open('params_dailyv70.pkl', 'rb'))
mu = params['mu']
std = params['std']
features = params['features']

In [24]:
#ib = IB()
#connection = ib.connect()

In [25]:
symbol = "EURUSD=X"
start = "2024-01-01"
end = "2024-08-27"
interval = '1d'


def prepare_dataframe():
    df = yf.download(symbol, start, end, interval=interval)
    #df.rename(columns={"Close": "price"}, inplace=True)
    df.drop(columns=['Adj Close', 'Volume'], inplace=True)
    #df.index = df.index.tz_convert('utc')
    df['returns'] = np.log(df['Close'].div(df['Close'].shift(1)))
    df['day_of_week'] = df.index.dayofweek
    df['dir'] = np.where(df['returns'] > 0, 1,0)
    df.dropna(inplace=True)
    
        
    ##INSERT TODAY
    df.loc[pd.to_datetime('today').normalize()] = np.nan
    ####
    
    tnx_symbol = "^TNX"
    
    tnx = yf.download(tnx_symbol, start, end, interval=interval)
    #df.rename(columns={"Close": "price"}, inplace=True)
    tnx.drop(columns=['Adj Close', 'Open', 'High', 'Low'], inplace=True)
    #df.index = df.index.tz_convert('utc')
    tnx.rename(columns={'Close': 'tnx_close', 'Volume': 'tnx_volume'}, inplace=True)
    tnx['tnx_returns'] = np.log(tnx['tnx_close'].div(tnx['tnx_close'].shift(1)))
    #tnx['tnx_simple_returns'] = tnx['tnx_close'] - tnx['tnx_close'].shift(1)
    tnx.dropna(inplace=True)
    forex_combined = tnx.join(df, how='outer')
    
    # snp_symbol = "^SPX"
    # 
    # snp = yf.download(snp_symbol, start, end, interval=interval)
    # #df.rename(columns={"Close": "price"}, inplace=True)
    # snp.drop(columns=['Adj Close', 'Open', 'High', 'Low'], inplace=True)
    # #df.index = df.index.tz_convert('utc')
    # snp.rename(columns={'Close': 'snp_close', 'Volume': 'snp_volume'}, inplace=True)
    # snp['snp_returns'] = np.log(snp['snp_close'].div(snp['snp_close'].shift(1)))
    # snp.dropna(inplace=True)
    # forex_combined = snp.join(forex_combined, how='outer')
    
    gold_symbol = "GC=F"
    
    gold = yf.download(gold_symbol, start, end, interval=interval)
    #df.rename(columns={"Close": "price"}, inplace=True)
    gold.drop(columns=['Adj Close', 'Open', 'High', 'Low'], inplace=True)
    #df.index = df.index.tz_convert('utc')
    gold.rename(columns={'Close': 'gold_close', 'Volume': 'gold_volume'}, inplace=True)
    gold['gold_returns'] = np.log(gold['gold_close'].div(gold['gold_close'].shift(1)))
    gold.dropna(inplace=True)
    forex_combined = gold.join(forex_combined, how='outer')
    
    oil_symbol = "CL=F"
    
    oil = yf.download(oil_symbol, start, end, interval=interval)
    #df.rename(columns={"Close": "price"}, inplace=True)
    oil.drop(columns=['Adj Close', 'Open', 'High', 'Low'], inplace=True)
    oil.rename(columns={'Close': 'oil_close', 'Volume': 'oil_volume'}, inplace=True)
    oil['oil_returns'] = np.log(oil['oil_close'].div(oil['oil_close'].shift(1)))
    oil.dropna(inplace=True)
    forex_combined = oil.join(forex_combined, how='outer')
    
    indexes = [
        {"yfinance_ticker": "^SPX", "name": "sp_500"},
    #    {"yfinance_ticker": "URTH", "name": "msci_world_index"},
     #   {"yfinance_ticker": "^NDX", "name": "nasdaq_100"},
      #  {"yfinance_ticker": "ACWI", "name": "ftse_all_world_index"},
        {"yfinance_ticker": "^W5000", "name": "wilshire_5000_total_market_index"},
        {"yfinance_ticker": "^N225", "name": "nikkei_225"},
      #  {"yfinance_ticker": "^STOXX50E", "name": "euro_stoxx_50"},
      #  {"yfinance_ticker": "000001.SS", "name": "shanghai_composite_index"},
      #   {"yfinance_ticker": "^HSI", "name": "hang_seng_index"},
      #  {"yfinance_ticker": "EEM", "name": "msci_emerging_markets_index"},
    #    {"yfinance_ticker": "^FTSE", "name": "ftse_100"},
        {"yfinance_ticker": "^GDAXI", "name": "dax"},
        {"yfinance_ticker": "^FCHI", "name": "cac_40"},
     #   {"yfinance_ticker": "^GSPTSE", "name": "sp_tsx_composite_index"},
      #  {"yfinance_ticker": "^AXJO", "name": "asx_200"},
      #  {"yfinance_ticker": "^BSESN", "name": "bse_sensex"},
       #  {"yfinance_ticker": "^SSMI", "name": "swiss_market_index"},
     #   {"yfinance_ticker": "^KS11", "name": "kospi"},
        {"yfinance_ticker": "^IBEX", "name": "ibex_35"},
        {"yfinance_ticker": "^RUT", "name": "russell_2000"}
    ]
    
    
    def add_index(index_symbol, name):
        index_df = yf.download(index_symbol, start, end, interval=interval)
        index_df.drop(columns=['Adj Close', 'Open', 'High', 'Low'], inplace=True)
        index_df.rename(columns={'Close': f"{name}_close", 'Volume': f"{name}_volume"}, inplace=True)
        index_df[f"{name}_returns"] = np.log(index_df[f"{name}_close"].div(index_df[f"{name}_close"].shift(1)))
        index_df.dropna(inplace=True)
        return index_df
    
    combined_indexes_list = pd.DataFrame()
        
        
    features = ['Close', 'returns', 'oil_returns', 'tnx_returns', 'gold_returns', 'gold_volume', 'tnx_volume',]
    
    for index in indexes:
        try:
            df = add_index(index['yfinance_ticker'],  index['name'])
            combined_indexes_list = df.join(combined_indexes_list, how='outer')
      #      features.append(f"{index['name']}_volume")
            features.append(f"{index['name']}_returns")
        except Exception as error:
            print(error)
            print(index)
            
    forex_combined = combined_indexes_list.join(forex_combined, how='outer')
    
    
    forex_clean = forex_combined.copy()
    forex_clean.drop(columns=['Open', 'High', 'Low'], inplace=True)
    
    
    cols = []
    for f in features:
        for lag in range(1, 7 + 1):
            col = "{}_lag_{}".format(f, lag)
            forex_clean[col] = forex_clean[f].shift(lag)
        #    cols.append(col)
 #   forex_clean.dropna(inplace=True)
    
    df = forex_clean.copy()
    
    return df

In [26]:
df = prepare_dataframe()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [27]:
df_standardized = (df - mu) / std
df_standardized

,russell_2000_close,russell_2000_volume,russell_2000_returns,ibex_35_close,ibex_35_volume,ibex_35_returns,cac_40_close,cac_40_volume,cac_40_returns,dax_close,...,ibex_35_returns_lag_5,ibex_35_returns_lag_6,ibex_35_returns_lag_7,russell_2000_returns_lag_1,russell_2000_returns_lag_2,russell_2000_returns_lag_3,russell_2000_returns_lag_4,russell_2000_returns_lag_5,russell_2000_returns_lag_6,russell_2000_returns_lag_7
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-03,2.895318,0.199168,-1.626649,-0.027602,-0.035740,-0.842955,3.670347,-1.016574,-1.112277,3.030978,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-04,2.891083,0.026331,-0.042337,0.038345,-0.220584,0.850532,3.717812,-1.226386,0.392225,3.059127,...,NaN,NaN,NaN,-1.618401,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-05,2.872095,0.121014,-0.200601,0.029194,0.024806,-0.114182,3.681157,-1.369573,-0.265480,3.050893,...,NaN,NaN,NaN,-0.028711,-1.666667,NaN,NaN,NaN,NaN,NaN
2024-01-08,2.981210,0.046048,1.163633,0.051943,-0.226218,0.293259,3.717335,-1.483179,0.303788,3.094514,...,NaN,NaN,NaN,-0.187512,-0.041953,-1.655318,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-21,3.504311,-0.411350,0.797345,0.515051,-0.538383,0.165120,3.808518,-1.500492,0.390654,3.712640,...,0.209598,0.512211,0.057024,-0.694830,0.739594,0.211443,1.569790,-0.307721,1.032157,-0.562240
2024-08-22,3.445158,-0.512827,-0.570813,0.536215,-0.491200,0.249997,3.807771,-1.592699,0.015243,3.728496,...,0.843053,0.199931,0.509312,0.813821,-0.722748,0.762393,0.215068,1.602598,-0.305849,1.035006
2024-08-23,3.643017,-0.284801,1.901231,0.598481,-0.470480,0.724772,3.872572,-1.468379,0.519796,3.778343,...,0.418966,0.855435,0.195062,-0.558981,0.819141,-0.706907,0.780232,0.225390,1.558675,-0.313733


In [29]:
df["probability"] = model.predict(df_standardized[features])
df.iloc[-30:][['probability']]

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1000us/step


,probability
Date,
2024-07-17,0.495676
2024-07-18,0.495676
2024-07-19,0.495676
2024-07-22,0.495676
2024-07-23,0.495676
2024-07-24,0.495676
2024-07-25,0.501693
2024-07-26,0.544537
2024-07-29,0.515394
